In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

import matplotlib as mpl
import numpy as np
import pandas as pd

from rcpl.config import BASE_DIR, MPL_RC, CONFIGS_DIR
from rcpl.prediction import ChainPredictor, format_float

mpl.rcParams.update(MPL_RC)
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [116]:
DEVICE = 'cuda'
D1 = 'exp2023-11-23.maftr2_4.template'
D2 = 'maftr2_4.template'

In [94]:
exp_path, e_id = BASE_DIR / 'data/epsp_stress/measured/2023-11-23.json', 1

In [95]:
gru_predictor = ChainPredictor(config_path=CONFIGS_DIR / 'maftr/maftr2_4.best/gru.yaml', predicts_scaled_theta=True)

In [105]:
unscaled_theta_opt_gru, stress_pred_opt_gru, _ = gru_predictor.predict_from_json_simplex(exp_path, verbose=True)

Origin score: 2441.20, Nelder-Mead score: 75.75


In [19]:
fraction = 1
validate_crlb_opt = gru_predictor.chain.validate_crlb.force().value(exp_path, crop_signal=[0, 681], unscaled_theta=unscaled_theta_opt_gru, fraction=fraction)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 260.09it/s]


In [20]:
unscaled_theta_prediction_dict = {f'${key}$': val for key, val in zip(gru_predictor.chain.dataset_info.value['latex_labels'], unscaled_theta_opt_gru)}

In [21]:
df_params = pd.DataFrame(unscaled_theta_prediction_dict.items(), columns=['parameter', r'$\widehat{\boldsymbol\theta}$'])
df_params['ESTD'] = [i / fraction for i in validate_crlb_opt['theta_std']]
df_params[r'$\sqrt{\mathrm{CRLB}}$'] = [np.sqrt(i) for i in validate_crlb_opt['crlb']['std']]
df_params[r'ESTD/$\sqrt{\mathrm{CRLB}}$'] = df_params['ESTD'] / df_params[r'$\sqrt{\mathrm{CRLB}}$']
df_params

,parameter,$\widehat{\boldsymbol\theta}$,ESTD,$\sqrt{\mathrm{CRLB}}$,ESTD/$\sqrt{\mathrm{CRLB}}$
0,$k_0$,210.460968,2.107498,0.767538,2.745791
1,$\kappa_1$,32330.250000,250.701294,16.561514,15.137583
2,$\kappa_2$,0.022078,0.000291,0.010531,0.027630
3,$c_1$,22260.906250,184.428925,12.541408,14.705599
4,$c_2$,2293.304199,34.663639,2.527423,13.715013
5,$c_3$,83.808731,32.147350,2.144702,14.989189
6,$c_4$,722.140564,95.224464,10.625799,8.961629
7,$a_1$,64.952286,2.420475,0.456718,5.299710
8,$a_2$,138.280243,2.963132,0.384041,7.715671
9,$a_3$,46.960052,2.989753,2.187723,1.366605


In [23]:
for col in df_params.columns[1:]:
    df_params[col] = df_params[col].apply(lambda x: format_float(x))
df_params = df_params.rename(columns={'parameter': ' '})
df_params

,,$\widehat{\boldsymbol\theta}$,ESTD,$\sqrt{\mathrm{CRLB}}$,ESTD/$\sqrt{\mathrm{CRLB}}$
0,$k_0$,210.5,2.107,0.7675,2.746
1,$\kappa_1$,3.233 $\times 10^{4}$,250.7,16.56,15.14
2,$\kappa_2$,0.02208,0.291 $\times 10^{-3}$,0.01053,0.02763
3,$c_1$,2.226 $\times 10^{4}$,184.4,12.54,14.71
4,$c_2$,2293,34.66,2.527,13.72
5,$c_3$,83.81,32.15,2.145,14.99
6,$c_4$,722.1,95.22,10.63,8.962
7,$a_1$,64.95,2.420,0.4567,5.300
8,$a_2$,138.3,2.963,0.384,7.716
9,$a_3$,46.96,2.990,2.188,1.367
